In [ ]:
class YoloLossCircleMultiBBox(nn.Module):
  """
  Custom loss function for YOLOv1 model with circle bounding boxes.

  This loss function computes the YOLOv1 loss for a set of predictions and ground truth targets.
  It is specifically designed for models predicting bounding boxes in the form of circles, where each
  bounding box is represented by its center coordinates (x, y) and radius (r), along with confidence
  scores and class probabilities.

  Args:
      C (int): Number of classes. Default is 1.
      S (int): Size of the grid. Default is 7.
      B (int): Number of bounding boxes per grid cell. Default is 3.
      lambda_noobj (float): Lambda coefficient for no object confidence loss. Default is 0.5.
      lambda_coord (float): Lambda coefficient for bounding box coordinates loss. Default is 5.

  Attributes:
      mse (nn.MSELoss): Mean squared error loss function.
      S (int): Size of the grid.
      B (int): Number of bounding boxes per grid cell.
      C (int): Number of classes.
      lambda_noobj (float): Lambda coefficient for no object confidence loss.
      lambda_coord (float): Lambda coefficient for bounding box coordinates loss.
  """
  def __init__(self, C: int = 1, S: int = 7, B : int = 3, lambda_noobj: float = 0.5, lambda_coord: float = 5):
    super().__init__()
    self.mse = nn.MSELoss(reduction="sum")
    self.S = S
    self.B = B
    self.C = C
    self.lambda_noobj = lambda_noobj
    self.lambda_coord = lambda_coord

  def forward(self, predictions, target):
    """
    Computes the YOLOv1 loss for a set of predictions and ground truth targets.

    Args:
        predictions (tensor): Predictions of the model (BATCH, S, S, C+4*B)
        target (tensor): Ground truth targets (BATCH, S, S, C+4*B)

    Returns:
        float: Total loss value.
    """
    predictions = predictions.reshape(-1, self.S, self.S, self.C + 4 * self.B)

    #bounding box predictor to be responsible for each object
    ious = []
    for b in range(self.B):
        ious.append(intersection_over_union(predictions=torch.cat([predictions[..., (self.C + 1 + b*4):(self.C + 3 + b*4)],
                                                                  (predictions[..., (self.C + 3 + b*4):(self.C + 4 + b*4)]*self.S)], dim = -1),
                                                 labels=torch.cat([target[..., (self.C + 1):(self.C + 3)],
                                                                  (target[..., (self.C + 3):(self.C + 4)]*self.S)], dim = -1))
                    ) # -> tensor(BATCH, S, S, 1)
    #Cheaking witch bbox have highest IoU value
    box_indexes = torch.argmax(torch.stack(ious), dim=0)

    ij_1_obj = target[..., self.C:(self.C+1)] #If an object is present in grid cell "i" and the "j"th bounding box predictor is “responsible” for that prediction

    #Travel throught all bboxes and selects that ones with the biggest IoU score
    temp_pred_pxyr = predictions[..., (self.C):(self.C+4)]
    for b in range(1, self.B):
        temp_pred_pxyr = torch.where(box_indexes == (b), predictions[..., (self.C + 4 * b):(self.C + 4 + 4 * b)], temp_pred_pxyr)

    matter_predictions = ij_1_obj * temp_pred_pxyr[..., 1 : 4] # torch.Size([S, S, 3]) == torch.Size([S, S, 1]) * torch.Size([S, S, 3])) ← 1ob*Σ(S²)Σ(B)(x',y',r')
    box_tar = ij_1_obj * target[..., self.C+1 : self.C+4] #(torch.Size([S, S, 3]) == (torch.Size([S, S, 1]) * torch.Size([S, S, 3]) ← 1ob*Σ(S²)Σ(B)(x,y,r)

    #Calculate root from r
    box_tar[..., 2:3] = torch.sqrt(torch.abs(box_tar[..., 2:3] + 1e-6))  # 1Σ(S²)Σ(B)(r) → Σ(S²)Σ(B)(√r)
    matter_predictions[..., 2:3] = torch.sign(matter_predictions[..., 2:3]) * torch.sqrt(torch.abs(matter_predictions[..., 2:3] + 1e-6)) # Σ(S²)Σ(B)(r') → Σ(S²)Σ(B)(√r')

    #Calculate loss for coordinates == λcord(1ob*Σ(S²)Σ(B)[(x-x')²+(y-y')²+(√r-√r')²])
    L1 = self.lambda_coord*self.mse(box_tar, matter_predictions)

    ij_1_noobj = 1 - ij_1_obj
    obj_confidence_pred_box = ij_1_obj * temp_pred_pxyr[..., 0 : 1] # torch.Size([S, S, 1]) * (torch.Size([S, S, 1]) == #torch.Size([S, S, 1])  ←  1ob*Σ(S²)Σ(B)(Č)
    obj_confidence_ground_box  = ij_1_obj * target[..., self.C:(self.C+1)] #torch.Size([S, S, 1]) * -||-  == #torch.Size([S, S, 1])  ←  1ob*Σ(S²)Σ(B)(C)
    no_obj_confidence_pred_box = ij_1_noobj * temp_pred_pxyr[..., 0 : 1] #torch.Size([S, S, 1]) * -||-  == #torch.Size([S, S, 1])  ←  1noob*Σ(S²)Σ(B)(Č)
    no_obj_confidence_ground_box = ij_1_noobj * target[..., self.C:(self.C+1)] #torch.Size([S, S, 1]) * -||-  == #torch.Size([S, S, 1])  ←  1noob*Σ(S²)Σ(B)(C)

    #Calculate prediction loss == 1ob*Σ(S²)Σ(B)(C-Č)² + λ1noob(1noob*Σ(S²)Σ(B)(C-Čbbox))²
    L2 = self.mse(obj_confidence_ground_box, obj_confidence_pred_box) + self.lambda_noobj * self.mse(no_obj_confidence_ground_box, no_obj_confidence_pred_box)

    prediction_network = ij_1_obj * predictions[..., :self.C]  #torch.Size([S, S, 1]) * -||-  -> #torch.Size([S, S, 1]) == 1ob*Σ(S²)Σ(B)(p')
    the_class = ij_1_obj * target[..., :self.C]#torch.Size([S, S, 1]) * -||-  -> #torch.Size([S, S, 1]) == 1ob*Σ(S²)Σ(B)(p)

    #Calculate clasification loss == 1ob*Σ(S²)Σ(B)(p-p')²
    L3 = self.mse(the_class, prediction_network)

    #Return sum of 3 loss:  λcord(1ob*Σ(S²)Σ(B)[(x-x')²+(y-y')²+(√r-√r')²]) +  1ob*Σ(S²)Σ(B)(C-Č)² + λ1noob(1noob*Σ(S²)Σ(B)(C-Čbbox))² + 1ob*Σ(S²)Σ(B)(p-p')²
    return L1+L2+L3